In [7]:
import pandas as pd
from surprise import prediction_algorithms as pa
from surprise import Dataset, Reader
from surprise import evaluate, print_perf

In [8]:
data = pd.read_csv('./movielens_small/ratings.csv')

data = data.sort_values(['timestamp'], ascending=[1])
number_of_rows = len(data)


In [9]:

reader = Reader(rating_scale=(1, 5))
train_data = Dataset.load_from_df(data[['userId','movieId','rating']], reader)

algo = pa.knns.KNNBasic(k=10)

train_data.split(n_folds=5)

perf = evaluate(algo, train_data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm KNNBasic.

------------
Fold 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9600
MAE:  0.7349
------------
Fold 2
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9698
MAE:  0.7408
------------
Fold 3
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9703
MAE:  0.7434
------------
Fold 4
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9593
MAE:  0.7355
------------
Fold 5
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9731
MAE:  0.7464
------------
------------
Mean RMSE: 0.9665
Mean MAE : 0.7402
------------
------------


In [33]:
split = int(0.8*number_of_rows)
print(split)
train_data = data[:split]
test_data = data[split:]

reader = Reader(rating_scale=(1, 5))
train_data = Dataset.load_from_df(train_data[['userId','movieId','rating']], reader)
train_data = train_data.build_full_trainset()

algo = pa.knns.KNNBasic(k=10)

algo.train(train_data)

s = 0
for index,row in test_data.iterrows():
    ans = algo.predict(row['userId'],row['movieId'])
    s += (ans.est-row['rating'])**2
print((s/)**0.5)

80003
Computing the msd similarity matrix...
Done computing similarity matrix.
